# Krakenbot usage example

Run this cell in case of module import problems.

In [1]:
import sys
# change to path with krakenbot on your local machine
path = 'C:/Users/szgrad/repos/krakenbot/'
sys.path.append(path)
import krakenbot

Import required modules.

In [2]:
from krakenbot.database.connectors.sqlite import SQLiteConnector
from krakenbot.rest.bot import KrakenBot
import pandas as pd

Create the database connector and `KrakenBot` instances.

In [3]:
# Input your database name.
db_name = 'test.db'
connector = SQLiteConnector(db_name=db_name)
bot = KrakenBot(db_connector=connector)


Fetches the latest batch of transactions (up to 1000).

In [4]:
# Change the batch size (up to 1000 positions) and the tradable asset pair
size = 3
pair = 'XBTUSD'
recent_trades = bot.recent_trades(size=size, pair=pair)

Prepare column names.

In [5]:
trades = recent_trades[pair]
columns = ['"pair name"', 'price', 'volume', 'time', '"buy(b)/sell(s)"', '"market(m)/limit(l)"', 'miscellaneous']
columns_names = " ,".join(columns)

Create the example table.

In [6]:
# Rename the table if you want.
table_name = 'recent_trades'
query_table = f'create table if not exists {table_name}  ("trade_id" integer primary key autoincrement, {columns_names})'
connector.execute(query_table)

Fill the table with the recent trade data.

In [7]:
# Change the batch size (up to 1000 positions) and the tradable asset pair
size = 3
pair = 'XBTUSD'
bot.update_recent_trades(table_name=table_name, pair=pair, size=size)


Check the data inside the table.

In [8]:
connector.execute(f'select * from {table_name}')

[(0, 'XBTUSD', '20157.30000', '0.06272000', 1663242681.6749911, 's', 'l', ''),
 (1, 'XBTUSD', '20157.30000', '0.00149000', 1663242682.584622, 's', 'l', ''),
 (2, 'XBTUSD', '20144.60000', '0.00010000', 1663242727.5302083, 'b', 'm', '')]

Update the table with the newest data.

In [9]:
bot.update_recent_trades(table_name=table_name, pair=pair, size=size)
connector.execute(f'select * from {table_name}')

[(0, 'XBTUSD', '20144.60000', '0.00010000', 1663242727.5302083, 'b', 'm', ''),
 (1, 'XBTUSD', '20144.60000', '0.02208824', 1663242727.5336277, 'b', 'l', ''),
 (2, 'XBTUSD', '20146.40000', '0.50000000', 1663242728.9173088, 'b', 'm', '')]

Repeat this action every 20 seconds for a minute and save the log info into the log file.

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)
bot.repeat_action(action_name='update_recent_trades', duration=60, interval=20, table_name=table_name, size=size, pair=pair)

15:03:40,118 urllib3.connectionpool DEBUG Starting new HTTPS connection (1): api.kraken.com:443
15:03:40,459 urllib3.connectionpool DEBUG https://api.kraken.com:443 "GET /0/public/Trades?pair=XBTUSD HTTP/1.1" 200 None
15:03:40,500 krakenbot.rest.bot INFO Updated row: [(0, 'XBTUSD', '20144.60000', '0.00010000', 1663242727.5302083, 'b', 'm', '')]
15:03:40,503 krakenbot.rest.bot INFO Updated row: [(1, 'XBTUSD', '20144.60000', '0.02208824', 1663242727.5336277, 'b', 'l', '')]
15:03:40,506 krakenbot.rest.bot INFO Updated row: [(2, 'XBTUSD', '20146.40000', '0.50000000', 1663242728.9173088, 'b', 'm', '')]
15:03:40,507 krakenbot.rest.bot INFO The action update_recent_trades executed on date 2022-09-15 15:03:40.507454
15:04:00,517 urllib3.connectionpool DEBUG Starting new HTTPS connection (1): api.kraken.com:443
15:04:00,848 urllib3.connectionpool DEBUG https://api.kraken.com:443 "GET /0/public/Trades?pair=XBTUSD HTTP/1.1" 200 None
15:04:00,896 krakenbot.rest.bot INFO Updated row: [(0, 'XBTUSD',

Check our table changes.

In [11]:
select_recent_trades = connector.execute(f'select * from {table_name}')
select_recent_trades

[(0, 'XBTUSD', '20151.20000', '0.00716134', 1663242765.7147331, 'b', 'm', ''),
 (1, 'XBTUSD', '20151.10000', '0.00446611', 1663242770.6794298, 'b', 'l', ''),
 (2, 'XBTUSD', '20151.20000', '0.00380413', 1663242770.6811762, 'b', 'l', '')]